In [1]:
import torch
from torchvision import models, transforms
from PIL import Image

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from PIL import Image

# EfficientNet-B0 모델 불러오기
model = models.efficientnet_b0(pretrained=True)

# 출력층을 회귀용으로 변경 (1개의 뉴런, 활성화 함수 없음)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 1)  # 출력층: 1개 숫자 (점수)

# 모델 평가 모드로 전환
model.eval()

# 이미지 전처리 함수 정의
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 이미지 불러오기 및 변환
img_path = "./images/image_6.jpg"
img = Image.open(img_path)
img = transform(img).unsqueeze(0)

# 예측 수행
with torch.no_grad():
    predicted_score = model(img).item()  # 점수 출력

# 점수 범위 조정 (0~5점 범위로 제한)
predicted_score = max(0, min(5, predicted_score))

# 결과 출력
print(f"예측된 방 사진 점수: {predicted_score:.2f}/5")


예측된 방 사진 점수: 0.12/5


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image

# 데이터셋 클래스 정의
class AirbnbDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 0]
        score = self.data.iloc[idx, 1]

        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(score, dtype=torch.float32)

# 전처리 정의
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 데이터 로드
dataset = AirbnbDataset(csv_file="airbnb_scores.csv", transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# EfficientNet 불러오기 (특징 추출 방식)
model = models.efficientnet_b0(pretrained=True)

# 모든 합성곱 층을 Freeze (고정) -> 마지막 레이어만 학습
for param in model.features.parameters():
    param.requires_grad = False

# 출력층 변경 (1개 뉴런, 회귀 모델)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 1)

# 손실 함수 및 옵티마이저
criterion = nn.MSELoss()  # 평균제곱오차(MSE)
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)

# 학습 루프 (5 Epoch)
for epoch in range(5):
    model.train()
    epoch_loss = 0

    for images, scores in dataloader:
        optimizer.zero_grad()
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, scores)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

print("특징 추출 방식 학습 완료!")

## 데이터 적을 때 사용(수천 개 이하)


In [ ]:
# EfficientNet 불러오기
model = models.efficientnet_b0(pretrained=True)

# 모든 레이어 학습 가능하도록 설정 (Fine-Tuning)
for param in model.parameters():
    param.requires_grad = True

# 출력층 변경 (1개 뉴런, 회귀 모델)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 1)

# 옵티마이저 및 손실 함수
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  #  학습률 조정

# 학습 루프 (5 Epoch)
for epoch in range(5):
    model.train()
    epoch_loss = 0

    for images, scores in dataloader:
        optimizer.zero_grad()
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, scores)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

print("Fine-Tuning 학습 완료!")


#  데이터 많을 때 사용 (수만 개 이상)